# Retreive Phoenix, AZ Zillow data from Quandl

In [26]:
#Import Dependencies
import requests
import pandas as pd
import quandl

#Get Quandl API key
quandl.read_key()
api_key = quandl.ApiConfig.api_key

#Constants
zillow_codes_file = "Phoenix_Zillow_Codes.xlsx"
zillow_data_file = "Zillow_data.csv"
zillow_pivot_file = "Zillow_pivot.csv"

In [12]:
def make_url(code):
    return f"https://www.quandl.com/api/v3/datasets/ZILLOW/{code}.json?api_key={api_key}"

In [13]:
def read_zillow_codes(codes_file):
    Zillow_data_df = pd.read_excel(codes_file)
    codesdf = Zillow_data_df.loc[:,["code","description"]]
    codesdf["url"] = codesdf["code"].apply(make_url)
    return codesdf

In [14]:
def get_responses(code_urls):
    datasets = []
    for url in code_urls:
        response = requests.get(url).json()
        responsedf = pd.DataFrame(response)
        datasets.append(responsedf)
    return responsedf

In [15]:
def get_dataset(index):
    code_value = codesdf["code"][index]
    code_label = codesdf["description"][index]
    data = quandl.get(f"ZILLOW/{code_value}", authtoken=api_key)
    data.reset_index(level=0, inplace=True)
    data["Code"]= code_value
    data["Label"] = code_label
    return data

## Read Zillow Codes

In [17]:
codesdf = read_zillow_codes(zillow_codes_file)
codesdf.head()

,code,description,url
0,C36159_BSICG,median market value of buyer seller index cross,https://www.quandl.com/api/v3/datasets/ZILLOW/...
1,C36159_BSICT,median market value of buyer seller index cros...,https://www.quandl.com/api/v3/datasets/ZILLOW/...
2,C36159_DOZ,median market value of days on zillow,https://www.quandl.com/api/v3/datasets/ZILLOW/...
3,C36159_FOR10K,"median market value of foreclosures per 10,000...",https://www.quandl.com/api/v3/datasets/ZILLOW/...
4,C36159_MDLNSAH,median market value of median daily listings -...,https://www.quandl.com/api/v3/datasets/ZILLOW/...


## Get Quandl responses for Zillow codes

In [19]:
responsedf = get_responses(codesdf["url"])
num_codes = len(codesdf)
print(f"Number of datasets: {num_codes}")
responsedf.head()

Number of datasets: 25


,dataset
collapse,None
column_index,None
column_names,"[Date, Value]"
data,"[[2019-07-31, 0.98004900245012], [2019-06-30, ..."
database_code,ZILLOW


## Make raw Zillow dataframe with all datasets appended 

In [23]:
# Get first dataset
data = get_dataset(0)
# Append remaining datasets
for code_index in range(1,num_codes):
    new_data = get_dataset(code_index)
    #print(new_data)
    data = data.append(new_data)
    print(f"{code_index}: {codesdf['code'][code_index]} {len(new_data)} {len(data)}")
# Save Raw Zillow Data
data.to_csv(zillow_data_file)
data.head()

1: C36159_BSICT 106 212
2: C36159_DOZ 115 327
3: C36159_FOR10K 130 457
4: C36159_MDLNSAH 80 537
5: C36159_MDLSAAH 80 617
6: C36159_MLNSAH 80 697
7: C36159_MLSAAH 80 777
8: C36159_MTURN 128 905
9: C36159_NMLNSAH 80 985
10: C36159_NMLSAAH 80 1065
11: C36159_SC 138 1203
12: C36159_SCBT 138 1341
13: C36159_SCMT 138 1479
14: C36159_SCSA 138 1617
15: C36159_SCSABT 138 1755
16: C36159_SCSAMT 138 1893
17: C36159_SCSATT 138 2031
18: C36159_SCTT 138 2169
19: C36159_SP 138 2307
20: C36159_SPBT 138 2445
21: C36159_SPFS 122 2567
22: C36159_SPMT 138 2705
23: C36159_SPTT 138 2843
24: C36159_STLR 115 2958


,Date,Value,Code,Label
0,2010-11-30,5.31,C36159_BSICG,median market value of buyer seller index cross
1,2010-12-31,6.25,C36159_BSICG,median market value of buyer seller index cross
2,2011-01-31,6.88,C36159_BSICG,median market value of buyer seller index cross
3,2011-02-28,6.88,C36159_BSICG,median market value of buyer seller index cross
4,2011-03-31,6.56,C36159_BSICG,median market value of buyer seller index cross


## Pivot ZIllow data to one dataset per column

In [27]:
# Pivot raw Zillow data
data_pivot = data.pivot_table(index="Date",columns=["Code"],values=["Value"],aggfunc='mean')

# Save Pivoted zillow data
data_pivot.to_csv(zillow_pivot_file)

data_pivot.head()

Value                                                       \
Code       C36159_BSICG C36159_BSICT C36159_DOZ C36159_FOR10K C36159_MDLNSAH   
Date                                                                           
2008-01-31          NaN          NaN        NaN          8.17            NaN   
2008-02-29          NaN          NaN        NaN          9.26            NaN   
2008-03-31          NaN          NaN        NaN          9.89            NaN   
2008-04-30          NaN          NaN        NaN         13.97            NaN   
2008-05-31          NaN          NaN        NaN         14.80            NaN   

                                                                    \
Code       C36159_MDLSAAH C36159_MLNSAH C36159_MLSAAH C36159_MTURN   
Date                                                                 
2008-01-31            NaN           NaN           NaN          NaN   
2008-02-29            NaN           NaN           NaN          NaN   
2008-03-31            NaN           NaN           NaN       0.0032   
2008-04-30            NaN           NaN           NaN       0.0039   
2008-05-31            NaN           NaN           NaN       0.0042   

                           ...                                            \
Code       C36159_NMLNSAH  ... C36159_SCSABT C36159_SCSAMT C36159_SCSATT   
Date                       ...                                             
2008-01-31            NaN  ...           NaN           NaN           NaN   
2008-02-29            NaN  ...           NaN           NaN           NaN   
2008-03-31            NaN  ...         482.0         384.0         291.0   
2008-04-30            NaN  ...         487.0         349.0         260.0   
2008-05-31            NaN  ...         526.0         372.0         257.0   

                                                                      \
Code       C36159_SCTT C36159_SP C36159_SPBT C36159_SPFS C36159_SPMT   
Date                                                                   
2008-01-31         NaN       NaN         NaN         NaN         NaN   
2008-02-29         NaN       NaN         NaN         NaN         NaN   
2008-03-31       238.0  221500.0    156900.0         NaN    244300.0   
2008-04-30       314.0  212600.0    147400.0         NaN    240300.0   
2008-05-31       335.0  204600.0    137300.0         NaN    233500.0   

                                    
Code       C36159_SPTT C36159_STLR  
Date                                
2008-01-31         NaN         NaN  
2008-02-29         NaN         NaN  
2008-03-31    465500.0         NaN  
2008-04-30    428200.0         NaN  
2008-05-31    413900.0         NaN  

[5 rows x 25 columns]

## Pivot data cleaned manually to fix headers
Cleaned data saved in Zillow_pivot_cleaned.csv